In [ ]:
!pip install colpali-engine==0.2.2 -q
!pip install -U bitsandbytes -q
!pip install mteb transformers tqdm typer seaborn -q
!pip install pdf2image einops google-generativeai gradio -q

In [ ]:
# Run in terminal
sudo apt install poppler-utils

In [ ]:
import os
import gradio as gr
import torch
from pdf2image import convert_from_path
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoProcessor
 
from colpali_engine.models.paligemma_colbert_architecture import ColPali
from colpali_engine.trainer.retrieval_evaluator import CustomEvaluator
from colpali_engine.utils.colpali_processing_utils import (
    process_images,
    process_queries,
)

In [ ]:
model_name = "vidore/colpali"  # specify the adapter model name
 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16
)
 
retrieval_model = ColPali.from_pretrained(
    "google/paligemma-3b-mix-448",
    torch_dtype=torch.float16,  # set the dtype to bfloat16
    device_map="cuda",
    quantization_config=bnb_config,
).eval()  # set the device to cuda
 
vit_config = VIT_CONFIG["google/paligemma-3b-mix-448"]
 
retrieval_model.load_adapter(model_name)
paligemma_processor = AutoProcessor.from_pretrained(model_name)
device = retrieval_model.device